In [159]:
import asyncio
import pandas as pd
import numpy as np
from math import sqrt

data = pd.read_csv('../CSV/FullMain.csv')
risk_metrics = {}
# RTSI
rtsi_data = data[data['<TICKER>'] == 'RTSI'].copy()
rtsi_data['market_returns'] = (rtsi_data['<CLOSE>'] - rtsi_data['<OPEN>']) / rtsi_data['<OPEN>']
mean_rtsi_simple_returns = rtsi_data['market_returns'].mean()
rtsi_data['var market_returns'] = rtsi_data['market_returns'] - mean_rtsi_simple_returns
rtsi_data['var market_returns **2'] = rtsi_data['var market_returns'] ** 2
variance = rtsi_data['market_returns'].var()


for name in data['<TICKER>'].unique():
    data_by_ticker = data[data['<TICKER>'] == name].copy()
    data_by_ticker['DATE'] = pd.to_datetime(data['<DATETIME>'].str.split(' ').str[0]) # DATE
    data_by_ticker['YEAR'] = data_by_ticker['DATE'].dt.year
    data_by_ticker_f = data_by_ticker[ data_by_ticker['YEAR'] == 2022].copy()

    data_by_ticker_f['Log'] = data_by_ticker_f['<CLOSE>'] / data_by_ticker_f['<OPEN>']
    data_by_ticker_f['Log'] = np.log(data_by_ticker_f['Log'])

    mean_log_returns = data_by_ticker_f['Log'].mean()

    data_by_ticker_f['var'] = data_by_ticker_f['Log'] - mean_log_returns
    data_by_ticker_f['var'] = data_by_ticker_f['var'] ** 2
    variance_log_returns = data_by_ticker_f['var'].mean()

    std_variance_log_returns = sqrt(variance_log_returns)

    year_vol = data_by_ticker_f.shape[0]
    year_vol = sqrt(year_vol) * std_variance_log_returns
    print(f'Волатильность годичная {name}', year_vol)


    data_by_ticker_f['simple return'] = (data_by_ticker_f['<CLOSE>'] - data_by_ticker_f['<OPEN>']) / data_by_ticker_f['<OPEN>']

    mean_simle_returns = data_by_ticker_f['simple return'].mean()

    data_by_ticker_f['var simple'] = data_by_ticker_f['simple return'] - mean_simle_returns
    data_by_ticker_f['var simple ** 2'] = data_by_ticker_f['var simple'] ** 2
    variance_simple_returns = data_by_ticker_f['var simple ** 2'].mean()

    std_div = sqrt(variance_simple_returns)

    year_std_div = data_by_ticker_f.shape[0]
    year_std_div = sqrt(year_std_div) * std_div
    print(f'Стандартное отклонение доходности годичная {name}', year_std_div)

    print(' ')

    risk_metrics[name] = [year_vol, year_std_div]

metrics_risk_df = pd.DataFrame(risk_metrics)

Волатильность годичная AFLT 0.6581546981633041
Стандартное отклонение доходности годичная AFLT 0.6619167271948482
 
Волатильность годичная ALRS 0.6364877923681913
Стандартное отклонение доходности годичная ALRS 0.6313047148678181
 
Волатильность годичная GAZP 0.8411428384888624
Стандартное отклонение доходности годичная GAZP 0.841611873248338
 
Волатильность годичная GMKN 0.4617993097014655
Стандартное отклонение доходности годичная GMKN 0.4721414055489103
 
Волатильность годичная HYDR 0.44049712922098166
Стандартное отклонение доходности годичная HYDR 0.44156719737527944
 
Волатильность годичная IRAO 0.8889853371348038
Стандартное отклонение доходности годичная IRAO 0.8688014565998525
 
Волатильность годичная LKOH 0.7658825811823311
Стандартное отклонение доходности годичная LKOH 0.8000130185823467
 
Волатильность годичная MGNT 0.6655042208222225
Стандартное отклонение доходности годичная MGNT 0.6669341863807107
 
Волатильность годичная MOEX 0.602836502222775
Стандартное отклонение до

In [160]:
data = pd.read_csv('../CSV/FullMain.csv')

market_data = data[data['<TICKER>'] == 'RTSI'].copy()
market_data['Market_Returns'] = market_data['<CLOSE>'].pct_change()

beta_coefficients = {}

tickers = data['<TICKER>'].unique()

for ticker in tickers:
    if ticker != 'RTSI':
        ticker_data = data[data['<TICKER>'] == ticker].copy()
        ticker_data['Returns'] = ticker_data['<CLOSE>'].pct_change()
        merged_data = pd.merge(ticker_data, market_data, on='<DATETIME>')

        cov_matrix = np.cov(merged_data['Returns'][1:], merged_data['Market_Returns'][1:])
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        beta_coefficients[ticker] = beta

for ticker in tickers:
    if ticker != 'RTSI':
        risk_metrics[ticker].append(beta_coefficients[ticker])
    else:
        risk_metrics[ticker].append(1)

risk_metrics

{'AFLT': [0.6581546981633041, 0.6619167271948482, 0.575120997669777],
 'ALRS': [0.6364877923681913, 0.6313047148678181, 0.5215354191760829],
 'GAZP': [0.8411428384888624, 0.841611873248338, 0.8169594009255574],
 'GMKN': [0.4617993097014655, 0.4721414055489103, 0.39110653368036247],
 'HYDR': [0.44049712922098166, 0.44156719737527944, 0.2437727247478341],
 'IRAO': [0.8889853371348038, 0.8688014565998525, 0.7002496794173172],
 'LKOH': [0.7658825811823311, 0.8000130185823467, 0.7766672723277237],
 'MGNT': [0.6655042208222225, 0.6669341863807107, 0.5118536972143375],
 'MOEX': [0.602836502222775, 0.6073211302445223, 0.49654690813169267],
 'MTLR': [0.6893715711576828, 0.6807554967235044, 0.6581497480346259],
 'MTSS': [0.5451749013631547, 0.5523364006205025, 0.42628453768034785],
 'NLMK': [0.6110462047592101, 0.621589491117089, 0.4631038862480911],
 'ROSN': [1.0413140131392349, 1.1031417134603565, 0.9139614982582328],
 'RTSI': [0.6918723746054423, 0.7077515101276152, 1],
 'SBER': [0.8906743201

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.plot(df.index, df['OPEN'])
plt.xlabel('Индекс')
plt.ylabel('OPEN')
plt.title('Зависимость OPEN от индекса')
plt.grid(True)
plt.show()

In [183]:
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame.from_dict(risk_metrics, orient='index')

df.columns = ['year_vol', 'year_std_div', 'beta']

scaler = StandardScaler()

standardized_data = scaler.fit_transform(df)

standardized_df = pd.DataFrame(standardized_data, columns=df.columns)

standardized_df['RISK'] = 0.25 * standardized_df['year_vol'] + 0.25 * standardized_df['year_std_div'] +0.50* standardized_df['beta']
standardized_df['TICKER'] = [i for i in tickers]

standardized_df_for_csv = standardized_df[['TICKER', 'RISK']]
standardized_df_for_csv.to_csv('../CSV/RISKS.csv')